# Prediction Using Past Data
## Refactored to use the new Modular Architecture

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

# import from rootdir
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from core.data import LocalCSVLoader, ResearchBitcoinLoader
from indicators.volatility import MABands, EMABands
from dotenv import load_dotenv
from os import getenv
load_dotenv()


### Load Data

In [2]:
# Load API Data (NUPL)
RB_ = getenv("RESEARCHBITCOIN_API_TOKEN")
if not rb_api_token:
    raise ValueError("RESEARCHBITCOIN_API_TOKEN is missing from .env file")
nupl_loader = ResearchBitcoinLoader(token=rb_api_token, item="net_unrealized_profit_loss/net_unrealized_profit_loss")

try:
    df_nupl = nupl_loader.load()
    print(f"NUPL Data Loaded: {df_nupl.shape}")
except Exception as e:
    print(f"API Load failed (expected if token invalid): {e}")
    # Create dummy data if API fails so notebook runs
    dates = pd.date_range("2018-01-01", "2024-01-01", freq="D")
    df_nupl = pd.DataFrame({"net_unrealized_profit_loss": np.random.uniform(-0.5, 0.8, len(dates))}, index=dates)
    df_nupl.index.name = "date"

# Load Price Data (CSV)
price_loader = LocalCSVLoader("ohlcv/btc_1d_data_2018_to_2025.csv")
df_price = price_loader.load()

# Log Transform (Data Prep)
df_price["Close_log"] = np.log10(df_price["Close"])
print(f"Price Data Loaded: {df_price.shape}")

Fetching from https://api.researchbitcoin.net/v1/net_unrealized_profit_loss/net_unrealized_profit_loss...
API Load failed (expected if token invalid): Cannot save file into a non-existent directory: 'ohlcv'


FileNotFoundError: File not found: ohlcv/btc_1d_data_2018_to_2025.csv

### Analyze NUPL (MA bands)

In [ ]:
# Calculate Indicators
nupl_ma = MABands(window=100, column="net_unrealized_profit_loss")
nupl_bands = nupl_ma.calculate(df_nupl)

# Merge results
df_nupl_analyzed = df_nupl.join(nupl_bands)

# Generate Signals (Logic extracted from previous SignalSerie)
df_nupl_analyzed["signal"] = "HOLD"
df_nupl_analyzed.loc[
    df_nupl_analyzed["net_unrealized_profit_loss"] > df_nupl_analyzed["upper_2.6std"], "signal"] = "SELL"
df_nupl_analyzed.loc[
    df_nupl_analyzed["net_unrealized_profit_loss"] < df_nupl_analyzed["lower_2.6std"], "signal"] = "BUY"

# Plotting
plt.figure(figsize=(14, 6))
plt.plot(df_nupl_analyzed.index, df_nupl_analyzed["net_unrealized_profit_loss"], label="NUPL", color="#0072B2",
         alpha=0.6)
plt.plot(df_nupl_analyzed.index, df_nupl_analyzed["upper_3std"], color="red", linestyle="--", label="3 Sigma")
plt.plot(df_nupl_analyzed.index, df_nupl_analyzed["lower_3std"], color="red", linestyle="--")
plt.plot(df_nupl_analyzed.index, df_nupl_analyzed["upper_2.6std"], color="orange", linestyle=":", label="2.6 Sigma")
plt.plot(df_nupl_analyzed.index, df_nupl_analyzed["lower_2.6std"], color="orange", linestyle=":")
plt.plot(df_nupl_analyzed.index, df_nupl_analyzed["mean"], color="green", label="Mean")

# Plot Signals
buys = df_nupl_analyzed[df_nupl_analyzed["signal"] == "BUY"]
sells = df_nupl_analyzed[df_nupl_analyzed["signal"] == "SELL"]
plt.scatter(buys.index, buys["net_unrealized_profit_loss"], marker="^", color="green", s=100, zorder=5)
plt.scatter(sells.index, sells["net_unrealized_profit_loss"], marker="v", color="red", s=100, zorder=5)

plt.title("NUPL with Moving Average Bands")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Analyze Price (EMA bands)

In [ ]:
# Calculate Indicators
price_ema = EMABands(window=100, column="Close_log")  # Using Window instead of Alpha for this example
price_bands = price_ema.calculate(df_price)

df_price_analyzed = df_price.join(price_bands)

# Plotting
plt.figure(figsize=(14, 6))
plt.plot(df_price_analyzed.index, df_price_analyzed["Close_log"], label="Log Price", color="black", alpha=0.5)
plt.fill_between(df_price_analyzed.index, df_price_analyzed["upper_3std"], df_price_analyzed["lower_3std"],
                 color='gray', alpha=0.1, label="3 Std Dev")
plt.plot(df_price_analyzed.index, df_price_analyzed["mean"], color="blue", label="EMA Mean")

plt.title("BTC Price (Log) with EMA Bands")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()